In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import json
import joblib

data = pd.read_csv('/home/raj/Desktop/Github_Project/House_Price_Predictor/data/cleanned_bengaluru_house_prices.csv')

x = data.drop('price',axis=1)
y = data['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(x_train, y_train)

prediction = model.predict(x_test)

mse = mean_squared_error(y_test, prediction)
print(f"mean squared error : {mse}")

columns = {
    'data_columns' : [col.lower() for col in x.columns]
}

with open("columns.json", "w") as f:
    f.write(json.dumps(columns))

joblib.dump(model, '/home/raj/Desktop/Github_Project/House_Price_Predictor/model/house_price_pedictor.pkl')

print("Model saved successfully")

mean squared error : 1115.577349145158
Model saved successfully


In [3]:
import joblib
import json
import numpy as np

# 1. Load the Model and the Columns
model = joblib.load('/home/raj/Desktop/Github_Project/House_Price_Predictor/model/house_price_pedictor.pkl')
with open("columns.json", "r") as f:
    data_columns = json.load(f)['data_columns']

# 2. Define the Prediction Function
def predict_price(location, sqft, bath, bhk):
    """
    This function takes the house details and asks the model for a price.
    """
    
    # Create an empty array for our inputs (filled with zeros)
    # It must be the same length as the columns list
    x = np.zeros(len(data_columns))

    # Set the values for Size, Bath, and BHK
    # We use data_columns.index() to find where 'total_sqft', 'bath', etc are located
    try:
        x[data_columns.index('total_sqft')] = sqft
        x[data_columns.index('bath')] = bath
        x[data_columns.index('bhk')] = bhk
    except ValueError:
        print("Error: Column names for sqft/bath/bhk don't match exactly.")
        return None

    # Handle Location
    # We search for the location in our columns list
    # If we find it, we set that specific column to 1 (True)
    loc_index = -1
    try:
        loc_index = data_columns.index(location.lower()) # Use lower() because we saved them as lower
    except ValueError:
        # If location is not found, we just ignore it (it counts as 'other')
        loc_index = -1

    if loc_index >= 0:
        x[loc_index] = 1

    # Predict!
    # The model expects a list of rows, so we wrap x in []
    return model.predict([x])[0]

# ------------------------------------------------------------------
# 3. Test It! (Change these values to test different houses)
# ------------------------------------------------------------------

# Test 1: A big house in '1st Phase JP Nagar'
price1 = predict_price('1st Phase JP Nagar', 1000, 2, 2)
print(f"Price for 1st Phase JP Nagar (1000 sqft, 2 BHK, 2 Bath): {price1:.2f} Lakhs")

# Test 2: A luxury house in 'Indira Nagar'
price2 = predict_price('Indira Nagar', 2000, 3, 3)
print(f"Price for Indira Nagar (2000 sqft, 3 BHK, 3 Bath): {price2:.2f} Lakhs")

# Test 3: A house in a location the model doesn't know (e.g., 'My Village')
price3 = predict_price('My Village', 1200, 2, 2)
print(f"Price for Unknown Location (1200 sqft, 2 BHK, 2 Bath): {price3:.2f} Lakhs")

Price for 1st Phase JP Nagar (1000 sqft, 2 BHK, 2 Bath): 93.00 Lakhs
Price for Indira Nagar (2000 sqft, 3 BHK, 3 Bath): 240.79 Lakhs
Price for Unknown Location (1200 sqft, 2 BHK, 2 Bath): 72.54 Lakhs


/home/raj/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/raj/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/raj/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
